In [1]:
# Importing Libraries
import numpy as np
import pandas as pd

In [2]:
from fbref import BotScrap

## Here I will trying to filter data from FBREF

In [3]:
data = BotScrap()
match = data.get_match('https://fbref.com/en/matches/2e4383ca/Arsenal-Leeds-United-April-1-2023-Premier-League')

In [4]:
home = match['home'][0]
# home.rename(columns={'': 'Player ID'})
home

,Arsenal (4-3-3),Arsenal (4-3-3).1
0,1,Aaron Ramsdale
1,4,Ben White
2,5,Thomas Partey
3,6,Gabriel Dos Santos
4,8,Martin Ødegaard
5,9,Gabriel Jesus
6,11,Martinelli
7,16,Rob Holding
8,19,Leandro Trossard
9,34,Granit Xhaka
